# Model

In [218]:
from keras.models import Sequential, clone_model
from keras.layers import Dense, Activation, Input, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, Merge, Flatten, BatchNormalization, add
from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import load_model

import keras.backend as K
from IPython.display import clear_output
from keras import optimizers
import math

In [288]:
class ImprovedModel(Model):
    def __init__(self, inputs, outputs):
        super().__init__(inputs, outputs)

    def predict_single(self, inputs):
        inputs = [np.expand_dims(input, 0) for input in inputs]
        outputs = self.predict(inputs)
        return [output[0] for output in outputs]

def mean_pred(y_true, y_pred):
    return K.mean(K.equal(K.gather(y_true, K.argmax(y_true, axis=-1)), K.gather(y_true, K.argmax(y_pred, axis=-1))))


def residual_layer(input_block, filters, kernel_size):

    x = conv_layer(input_block, filters, kernel_size)

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)

    x = add([input_block, x])

    x = LeakyReLU()(x)

    return (x)

def conv_layer(x, filters, kernel_size):

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)
    x = LeakyReLU()(x)

    return (x)

def build_model():
    
    input_1 = Input((4,8,2))
    convnet = input_1

    #convnet = conv_layer(convnet, 75, (3, 3))
   # convnet = residual_layer(convnet, 75, (3, 3))
   # xonvnet = residual_layer(convnet, 75, (3, 3))
   # convnet = residual_layer(convnet, 75, (3, 3))
   # convnet = residual_layer(convnet, 75, (3, 3))
   # convnet = residual_layer(convnet, 75, (3, 3))
   # convnet = conv_layer(convnet, 2, (1, 1))
    #convnet = Conv2D(32, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    convnet = Flatten()(convnet)

    input_2 = Input((4,))

    policy_out = concatenate([convnet, input_2])
    #policy_out = Dense(64, activation='relu')(policy_out)
    #policy_out = Dense(128, activation='relu')(policy_out)
    policy_out = Dense(1024, activation='relu')(policy_out)
    policy_out = Dense(32, activation='sigmoid')(policy_out)

    value_out = concatenate([convnet, input_2])
    #value_out = Dense(64, activation='relu')(value_out)
    #value_out = Dense(128, activation='relu')(value_out)
    #value_out = Dense(256, activation='relu')(value_out)
    value_out = Dense(1, activation='tanh')(value_out)

    model = ImprovedModel(inputs=[input_1, input_2], outputs=[policy_out, value_out])

    adam = optimizers.Adam()
    model.compile(optimizer=adam,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    return model